In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'

from glob import glob

import pandas as pd
import plotnine as p
from tqdm import tqdm
import anndata

Populating the interactive namespace from numpy and matplotlib


C:\Users\vale\Anaconda3\lib\site-packages\anndata\_core\anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [12]:
ls

 Volume in drive C is NVMe Drive
 Volume Serial Number is 9ACC-A435

 Directory of C:\Users\vale\Google Drive\Analysis\200428-slideseq-comparison

03/05/2020  14:42    <DIR>          .
03/05/2020  14:42    <DIR>          ..
03/05/2020  14:40    <DIR>          .ipynb_checkpoints
29/04/2020  09:05            18,072 1. Convert data to H5AD.ipynb
03/05/2020  14:42           410,343 200503 - Add coordinates and combine files.ipynb
28/04/2020  22:54    <DIR>          dask-worker-space
29/04/2020  01:17        91,299,083 Puck_190921_19.digital_expression.h5ad
28/04/2020  20:25        20,177,479 Puck_190921_19.digital_expression.txt.gz
28/04/2020  20:25         1,617,528 Puck_190921_19_bead_locations.csv
29/04/2020  01:17        32,059,023 Puck_190921_21.digital_expression.h5ad
28/04/2020  20:25         8,123,377 Puck_190921_21.digital_expression.txt.gz
28/04/2020  20:25           576,132 Puck_190921_21_bead_locations.csv
29/04/2020  01:18       155,684,905 Puck_190926_01.digital_expression.h5

In [25]:
adatas = []


for fname in tqdm(glob('Puck_*.h5ad')):
    adata = anndata.read(fname)
    obs = pd.read_csv(fname.replace('.digital_expression.h5ad', '_bead_locations.csv'), index_col=0)

    adata.obs['xcoord'] = obs['xcoord']
    adata.obs['ycoord'] = obs['ycoord']

    adatas.append(adata)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  5.63it/s]


In [31]:
all_data = adatas[0].concatenate(adatas[1:], join='outer')

In [34]:
all_data

AnnData object with n_obs × n_vars = 389037 × 27015 
    obs: 'batch', 'filename', 'xcoord', 'ycoord'

In [36]:
all_data.obs

,batch,filename,xcoord,ycoord
ACGATCCAAAAACN-0,0,Puck_190921_19.digital_expression.txt.gz,4384.141026,2775.358974
CGGTCCTACCCCCT-0,0,Puck_190921_19.digital_expression.txt.gz,3940.031646,5275.734177
ACCTTAGACCCCAA-0,0,Puck_190921_19.digital_expression.txt.gz,3186.365079,774.920635
TCCTCCACTTTTTT-0,0,Puck_190921_19.digital_expression.txt.gz,2043.488889,4715.733333
ACCCGCTACCCCCT-0,0,Puck_190921_19.digital_expression.txt.gz,3187.634615,770.711538
...,...,...,...,...
ACAATACAGATACT-8,8,Puck_200115_08.digital_expression.txt.gz,3025.500000,5516.000000
GGTCAGGTTAACGT-8,8,Puck_200115_08.digital_expression.txt.gz,3069.700000,3211.900000
CATTATGGAATTCT-8,8,Puck_200115_08.digital_expression.txt.gz,3507.100000,2811.000000
GACTTTTCTTAAAG-8,8,Puck_200115_08.digital_expression.txt.gz,2155.000000,1264.300000


In [39]:
tissue = {
    'Puck_190921_19': 'mouse E15 brain',
    'Puck_190921_21': 'mouse hippocampus',
    'Puck_190926_01': 'unknown',
    'Puck_190926_02': 'unknown',
    'Puck_190926_03': 'mouse embryo Slide-seqV2',
    'Puck_190926_06': 'unknown',
    'Puck_191007_07': 'unknown',
    'Puck_191204_01': 'mouse hippocampus',
    'Puck_200115_08': 'mouse hippocampus'
}

In [41]:
all_data.obs['tissue'] = all_data.obs['filename'].str.split('.').str.get(0).map(tissue)

In [42]:
all_data

AnnData object with n_obs × n_vars = 389037 × 27015 
    obs: 'batch', 'filename', 'xcoord', 'ycoord', 'tissue'

In [43]:
all_data.write('stickels_et_al_2020_slide_seqV2.h5ad')

... storing 'filename' as categorical
... storing 'tissue' as categorical
